In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

In [39]:
headers = {
    'authority': 'www.amazon.com.tr',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6',
    'cache-control': 'max-age=0',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'ubid-acbtr=257-0578411-5371008; session-id=257-5458463-5574564; i18n-prefs=TRY; lc-acbtr=tr_TR; session-id-time=2082787201l; session-token="QzTUVZQ7sWkSRJTgKASFq9JHUfP7TMNuD9VYO/XvOv3yF5cDSF+BGpoq3K4E5w58DogdhIZ7TogvWDajmp/FzWADATrpMmo2cCZYkvlq2vFpd09IUmtZy8/j71ZJ9alNC9wsa42LlxkBESCtotKEOG9gX6fyJxZTt9JkpDL7U/GdU44OFE4sjvOcXDssQv2Vg5ADSuOXAMAv0y/1g2uczQ=="; csm-hit=tb:EYZQXR3CY8EMJHV7DJJ1+s-V05C79H7NXY17VENB5HJ|1652795008062&t:1652795008062&adb:adblk_no',
    'device-memory': '8',
    'downlink': '10',
    'dpr': '1',
    'ect': '4g',
    'referer': 'https://www.amazon.com.tr/elektronik/b/?ie=UTF8&node=12466497031&ref_=nav_cs_electronics',
    'rtt': '100',
    'sec-ch-device-memory': '8',
    'sec-ch-dpr': '1',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-viewport-width': '1920',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
    'viewport-width': '1920',
}

In [40]:
soup = BeautifulSoup(resp.text, "html.parser")

In [41]:
reviews = soup.find_all("div", {"class": "a-section review aok-relative"})

In [42]:
def get_review_stars(soup_obj: BeautifulSoup) -> str:
    try:
        review_str = soup_obj.find('a', {"class": "a-link-normal"}).get_text().split()[3]
        return review_str
    except Exception as e:
        return 'no_stars'
        print(e)

In [43]:
def get_review_body(soup_obj: BeautifulSoup) -> str:
    try:
        review_body = soup_obj.find('span', 
            {"class": "a-size-base review-text review-text-content"}
        ).get_text().strip()
        return review_body
    except Exception as e:
        return 'no_body'
        print(e)


In [44]:
def get_review_components(soup_obj: BeautifulSoup) -> dict:
    return {
        'stars': get_review_stars(soup_obj),
        'body': get_review_body(soup_obj), 
    }

In [45]:
records = [get_review_components(rev) for rev in reviews]
df = pd.DataFrame.from_records(records)
df.to_csv('amazon_980.csv')

In [46]:
ALL_REVIEWS = []

for x in range(8):
    u="https://www.amazon.com.tr/Samsung-NVMe-Dahili-Kat%C4%B1/product-reviews/B08GS7748F/ref=cm_cr_getr_d_paging_btm_prev_{btmNext}?ie=UTF8&reviewerType=all_reviews&pageNumber={pageNumber}".format(btmNext=x,pageNumber=x)
    
    resp = requests.get(u, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    reviews = soup.find_all("div", {"class": "a-section review aok-relative"})
    stars =soup.find_all("div",)
    records = [get_review_components(rev) for rev in reviews]
    ALL_REVIEWS.extend(records)
    
out = pd.DataFrame.from_records(ALL_REVIEWS)
out.head()    

,stars,body
0,"3,0",Şöyle hızlara çıkıyor falan demeyeceğim zaten ...
1,"5,0",2021 itibari ile paranın alabileceği en iyi m2...
2,"5,0",Asus pro gamer z97 anakartım için aldım. Önced...
3,"5,0",Öncelikle ürün PCI-e 4.0 ile en güncel teknolo...
4,"5,0",Amazon Prime farkıyla en uygun fiyata aynı gün...


In [36]:
from datetime import datetime


In [37]:
save_name = f"{datetime.now().strftime('%Y-%m-%d-%m')}.csv"
out.to_csv(save_name,index=False,mode='a')